O projeto deve conter:
- arquivo README.md, requirements.txt, diretórios notebook e arquivo app.py - FEITO
- link para acesso remoto (Render) - FEITO
- pelo menos um cabeçalho com texto - FEITO
- pelo menos 1 histograma - FEITO 
- pelo menos 1 gráfico de dispersão
- pelo menos um botão ou uma caixa de seleção - FEITO

DESAFIO: SUBSTITUIR BOTÕES POR CAIXA DE SELEÇÃO ONDE O USUÁRIO SELECIONA HIST OU DISPERÇÃO - FEITO

In [ ]:
# importando bibliotecas
import pandas as pd
import plotly.express as px
import streamlit as st
from pandas.api.types import is_numeric_dtype

#### Criação, edição e leitura do df

In [ ]:
# TESTES 
df_vehicles = pd.read_csv('../vehicles_us.csv') 
car_data = df_vehicles.copy()
# car_data.info()
# car_data.describe(include='all')
car_data.sample(10)

# car_data["transmission"].value_counts()
# car_data["cylinders"].value_counts()
# car_data["odometer"].describe().round(2)
car_data.sample(10)


In [ ]:
# VERSÃO PARA ADICIONAR AO APP.PY
@st.cache_data
def load_data():
    """Função para editar o df e remover colunas desnecessárias apenas ao carregar a página para evitar lentidão no site."""
    df = pd.read_csv("vehicles_us.csv")
    # dropando algumas colunas que não me interessam
    df = df.drop(columns=["days_listed", "date_posted"])
    # criando duas novas colunas, uma apenas com a marca e outra apenas com o nome do carro
    df[["brand", "model_name"]] = df["model"].str.split(pat=" ", n=1, expand=True)

    return df


car_data = load_data()

#### Carregamento e renderização da base de dados

In [ ]:
# EXIBINDO A TABELA DE DADOS
st.subheader("Base de dados")
st.write(car_data)
st.divider()

In [ ]:
# ALTERAÇÃO PARA MELHORAR A RENDERIZAÇÃO DA PÁGINA -> carregar a tabela apenas se o usuário desejar e dar ao usuário a opção de escolher a quantidade de dados para visualizar
st.subheader("Base de dados")

# botão de seleção para mostrar ou não a tabela
mostrar_tabela = st.checkbox("Mostrar tabela?", value=False)

if mostrar_tabela:
    # slider para escolher quantas linhas o usuário deseja exibir
    num_linhas = st.slider(
        "Quantidade de linhas a exibir:",
        min_value=10,
        max_value=len(car_data),
        value=50,
        step=10,
    )

    st.dataframe(car_data.head(num_linhas))
else:
    st.info("Marque a caixa acima para visualizar a tabela.")

#### Select box com opções de colunas do df para criação dos histogramas

In [ ]:
st.subheader("Histogramas")

feature_map = {
    "Ano de Fabricação": "model_year",
    "Tipo de Carro": "type",
    "Cor do Carro": "paint_color",
    "Marcas": "brand",
}

feature_label = st.selectbox(
    "Escolha uma variável para visualizar:",
    list(feature_map.keys()),
)

column_name = feature_map[feature_label]
series = car_data[column_name].dropna()

if is_numeric_dtype(series):
    fig = px.histogram(
        car_data,
        x=column_name,
        nbins=30,
        title=f"Histograma de {feature_label}",
    )
    fig.update_layout(xaxis_title=None)
else:
    fig = px.histogram(
        car_data,
        x=column_name,
        color=column_name,
        title=f"Distribuição de {feature_label}",
    )
    fig.update_layout(xaxis_title=None)

st.plotly_chart(fig, use_container_width=True)
st.caption("Os valores nulos foram removidos para geração dos gráficos acima.")
st.divider()

#### Botão que gera gráfico de dispersão

In [ ]:
def gerar_dispersao(df, x_col, y_col):
    """Função que cria um scatterplot baseado no clique do usuário em um botão e o adiciona à tela.
    Opções de gráfico:
    - quilometragem vs preço;
    - ano de fabricação vs preço.
    """
    fig = px.scatter(
        df,
        x=x_col,
        y=y_col,
        opacity=0.7,
        trendline="ols",
        # color=color_var,       # comente/retire se não quiser cores
        # size=size_var,         # comente/retire se não quiser tamanho variável
        title=f"{x_col} vs {y_col}"
    )
    
    # atualizando o eixo x da quilometragem de milhões para milhares 
    if x_col == "odometer":
        fig.update_xaxes(
            tickformat=",d",
            separatethousands=True
    )
    
    # exibindo no Streamlit:
    st.plotly_chart(fig, use_container_width=True)

st.subheader("Gráficos de dispersão")

# algorítmo abaixo não funcionou no streamlit, alterei para if para que o gráfico fosse renderizado na tela
# botao_odo_vs_price = st.button("Quilometragem vs Preço", on_click=lambda: gerar_dispersao(car_data, "odometer", "price"))
# botao_year_vs_price = st.button("Ano de Fabricação vs Preço", on_click=lambda: gerar_dispersao(car_data, "model_year", "price"))

# código atualizado -> o layout ficava estranho, com o gráfico aparecendo entre os botões, atualizei novamente (próximo código):
# if st.button("Quilometragem vs Preço"):
#     gerar_dispersao(car_data, "odometer", "price")    
# if st.button("Ano de Fabricação vs Preço"):
#     gerar_dispersao(car_data, "model_year", "price")    

# ultima atualização -> usando SESSION STATES:
# inicializando o session state
if "grafico_atual" not in st.session_state:
    st.session_state["grafico_atual"] = None

# adicionando e mantendo os botões lado a lado
button_1, button_2 = st.columns(2)

with button_1:
    if st.button("Quilometragem vs Preço"):
        st.session_state["grafico_atual"] = ("odometer", "price")
with button_2:
    if st.button("Ano de Fabricação vs Preço"):
        st.session_state["grafico_atual"] = ("model_year", "price")

# atualizando o session state para mostrar o gráfico na tela
if st.session_state["grafico_atual"] is not None:
    x_col, y_col = st.session_state["grafico_atual"]
    gerar_dispersao(car_data, x_col, y_col)
    

In [ ]:
# ALTERAÇÃO PARA MELHORAR A RENDERIZAÇÃO DA PÁGINA -> limitando a amostra de dados usados para criar os gráficos
st.subheader("Gráficos de dispersão")

def gerar_dispersao(df, x_col, y_col, max_points=10000):
    """Função que cria um scatterplot (com amostragem para melhorar o desempenho) baseado no clique do usuário em um botão e o adiciona à tela.
    Opções de gráfico:
    - quilometragem vs preço;
    - ano de fabricação vs preço.
    """
    df_plot = df[[x_col, y_col]].dropna()

    # amostragem se há muitos pontos
    if len(df_plot) > max_points:
        df_plot = df_plot.sample(max_points, random_state=42)

    fig = px.scatter(
        df_plot,
        x=x_col,
        y=y_col,
        opacity=0.7,
        title=f"{x_col} vs {y_col}",
        render_mode="webgl",
    )

    if x_col == "odometer":
        fig.update_xaxes(
            tickformat=",d",
            separatethousands=True,
        )

    st.plotly_chart(fig, use_container_width=True)
    
    
# inicializando o session state
if "grafico_atual" not in st.session_state:
    st.session_state["grafico_atual"] = None

# adicionando e mantendo os botões lado a lado
button_1, button_2 = st.columns(2)

with button_1:
    if st.button("Quilometragem vs Preço"):
        st.session_state["grafico_atual"] = ("odometer", "price")
with button_2:
    if st.button("Ano de Fabricação vs Preço"):
        st.session_state["grafico_atual"] = ("model_year", "price")

# atualizando o session state para mostrar o gráfico na tela 
if st.session_state["grafico_atual"] is not None:
    x_col, y_col = st.session_state["grafico_atual"]
    gerar_dispersao(car_data, x_col, y_col, max_points=10000)
    
    
st.caption("A amostragem dos dados foi limitada em 10,000 para otimizar a velocidade de renderização.")